# TinyMS SSD300 Tutorial

### In this tutorial, starting an SSD300 inference server and drawing bounding boxes using TinyMS API will be demonstrated. 

## Prerequisite
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.1`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## Introduction

TinyMS is a high-level API which is designed for amateur of deep learning. It minimizes the number of actions of users required to construct, train, evaluate and serve a model. TinyMS also provides tutorials and documentations for developers. 

In this tutorial, the ckpt files will be provided since training SSD300 on CPU is time consuming, so there are four steps to start a server and making predicts: `get the ckpt files`, `define servable json`, `start the server` and `making predicts` in which the server will be started in a subprocess.

In [ ]:
import os
import json
from PIL import Image
from tinyms.serving import start_server, predict, list_servables, shutdown, server_started
from tinyms.vision import ImageViewer

## Steps

### 1. Get the ckpt file

In this tutorial, we provide the SSD300 checkpoint file which is trained with [VOC](http://host.robots.ox.ac.uk/pascal/VOC/) dataset. A SSD300 ckpt file is required for the backend to run the prediction. We recommend downloading our pretrained checkpoint file, click [HERE](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com/resnet-50/ckpt_files/voc/ssd300.ckpt) to download the ckpt file and save this file to `/etc/tinyms/serving/ssd300/ssd300.ckpt`.

### Or run the following code to download and store the ckpt file:

In [ ]:
ssd300_ckpt_folder = '/etc/tinyms/serving/ssd300'
ssd300_ckpt_path = '/etc/tinyms/serving/ssd300/ssd300.ckpt'

if not os.path.exists(ssd300_ckpt_folder):
    !mkdir -p  /etc/tinyms/serving/ssd300
    !wget -P /etc/tinyms/serving/ssd300 https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com/resnet-50/ckpt_files/voc/ssd300.ckpt
else:
    print('ssd300_ckpt_folder already exists')
    if not os.path.exists(ssd300_ckpt_path):
        !wget -P /etc/tinyms/serving/ssd300 https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com/resnet-50/ckpt_files/voc/ssd300.ckpt
    else:
        print('ssd300 ckpt file already exists')

### 2. Define servable.json 

Run this code to define the servable json file for later use:

In [ ]:
servable_json = [{'name': 'ssd300', 
                  'description': 'This servable hosts a ssd300 model predicting bounding boxes', 
                  'model': {
                      "name": "ssd300", 
                      "format": "ckpt", 
                      "class_num": 21}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

### 3. Start server

#### 3.1 Introduction
TinyMS Serving is a C/S(client/server) structure. There is a server and client. TinyMS using [Flask](https://flask.palletsprojects.com/en/1.1.x/) which is a micro web framework written in python as the C/S communication tool. In order to serve a model, user must start server first. If successfully started, the server will be run in a subprocess and listening to POST requests from 127.0.0.1 port 5000 sent by client and handle the requests using MindSpore backend which will construct the model, run the prediction and send the result back to the client.  

#### 3.2 start server
Run the following code block to start the server:

In [ ]:
start_server()

### 4. Make predictions

#### 4.1 Upload the pic

A picture is required to be the input. In this tutorial, the SSD300 ckpt requires a picture containing objects of 
```
['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
```
Click [HERE](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com/tinyms-test-pics/ssd300_test/ssd300_test.jpeg) to download the picture which is used in this tutorial. Upload the pic, if using terminal, either `scp` or `wget` will do, if running in Jupyter, click `Upload` button at the top right and select the picture. Save the picture at the root folder, rename to `ssd300_test.jpeg`(or any name you want).

### Or run the following code to download the picture used in this tutorial:

In [ ]:
# download the test pic
if not os.path.exists('/root/ssd300_test.jpeg'):
    !wget -P /root/ https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com/tinyms-test-pics/ssd300_test/ssd300_test.jpeg
else:
    print('ssd300_test.jpeg already exists')

####  4.2 List servables

Now, use `list_servables` function to check what model is servable right now.

In [ ]:
list_servables()

If the output description shows it is a `ssd300` model, run the following code to predict the bounding boxes

#### 4.3 Sending request and get the result

Run `predict` function and get the result, right only `TOP1CLASS` strategy is supported. Call `ImageViewer.draw` to draw the bounding boxes

In [ ]:
# set image path and output strategy(only TOP1_CLASS)
image_path = "/root/ssd300_test.jpeg"
strategy = "TOP1_CLASS"

labels = ['background', 
          'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 
          'bus', 'car', 'cat', 'chair', 'cow', 
          'diningtable', 'dog', 'horse', 'motorbike', 'person', 
          'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

# predict(image_path, servable_name, dataset_name, strategy)
# ImageViewer(img, title)
# ImageViewer.draw(predict_result, labels)
if server_started() is True:
    res = predict(image_path, 'ssd300', 'voc', strategy)
    img_viewer = ImageViewer(Image.open(image_path))
    img_viewer.draw(res, labels)
else:
    print("Server not started")

## Check output

If the input picture is shown with bounding boxes labeled with object classes and score, that means the prediction is successfully performed.

## Shutdown server

In [ ]:
shutdown()